In [15]:
import pandas as pd
import numpy as np
import bar_chart_race as bcr

def map_columns(df, newcol):
    emissie_mapping = pd.read_csv('emissie_mapping.csv', sep=';')
    mapping = {k:v for k,v in zip(emissie_mapping['From'], emissie_mapping[newcol])}
    return df.rename(columns=mapping)

# read file and wrangle to format needed for bar chart plotter
file = 'Emissiecijfers+2013-2020.ods'
emissions = pd.read_excel(file, engine='odf')
emissions = emissions.T
emissions.columns = emissions.loc['Inrichtingsnaam',:]
emissions.fillna(0, inplace=True)
emissions.drop(0, axis=1,inplace=True)
emissions.replace('nan', 0, inplace=True)
emissions.replace('NaN', 0, inplace=True)
emissions.replace('-', 0, inplace=True)
filtered_emissions = emissions[4:]

# make numeric
cols = filtered_emissions.columns
for c in cols:
    filtered_emissions[c] = filtered_emissions[c].apply(pd.to_numeric, errors='coerce')
    


C:\Users\GEBRUI~1\AppData\Local\Temp/ipykernel_14552/1687757085.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_emissions[c] = filtered_emissions[c].apply(pd.to_numeric, errors='coerce')


In [16]:
# change index to an integer year
filtered_emissions.index = filtered_emissions.index.map(lambda x: int(x.replace(' ','')[-4:]))
# report in kton CO2
filtered_emissions /= 1000
# remove special character
filtered_emissions = filtered_emissions.rename(columns={'Warmte Station Galileïstraat':'Warmte Station Galileistraat'})
# Uniper centrale maasvlakte powerplant 3 is een kolencentrale, er worden sinds 2017 geen emissies meer gemeld. Ik vermoed dat dit komt doordat de emissies zijn samengevoegd onder Uniper Centrale Maasvlakte
filtered_emissions['Uniper Centrale Maasvlakte'] += filtered_emissions['Uniper Maasvlakte Powerplant 3']
filtered_emissions.drop('Uniper Maasvlakte Powerplant 3', axis=1, inplace=True)
filtered_emissions.head()

Inrichtingsnaam,"A.C. Hartman BV, locatie Sexbierum",A12/CPP Petrogas E&P Netherlands B.V.,Aardgasbuffer Zuidwending,Aardwarmte Centrale Den Haag,Abbott Healthcare Products BV,Abbott Laboratories B.V.,Academisch Medisch Centrum (AMC),Academisch Ziekenhuis Groningen,ADM Europoort B.V.,AGC Flat Glass Nederland BV,...,Wormdal Vastgoed BV,Yara Sluiskil B.V. BKG 1,Yara Sluiskil B.V. BKG 2,Yara Sluiskil B.V. BKG 3,Yara Sluiskil B.V. BKG 4,Yara Sluiskil B.V. BKG 5,Yara Sluiskil B.V. BKG 6,Zalco B.V.,Zeeland Refinery N.V.,Zwanenberg Food Oss B.V.
2013,32.544,63.616,5.273,0.000,14.230,14.829,41.633,24.781,175.286,2.156,...,0.202,698.597,1185.536,1224.587,62.839,140.411,228.464,2.767,1542.691,13.557
2014,27.027,64.574,3.919,0.000,13.366,14.525,36.620,20.646,168.287,0.277,...,0.177,713.142,1030.772,1213.616,77.750,154.782,223.178,7.949,1326.768,11.899
2015,27.178,63.192,2.777,0.036,13.485,15.186,37.052,20.939,156.023,0.081,...,0.071,772.982,1233.816,1312.062,77.725,141.980,223.841,12.881,1543.182,12.018
2016,27.158,66.617,3.573,0.211,12.830,15.554,38.047,20.688,151.672,0.000,...,0.073,730.962,1239.653,1301.474,70.074,160.705,226.529,11.832,1552.162,11.211
2017,28.942,68.273,2.236,0.141,12.632,15.670,37.856,16.042,149.157,0.000,...,0.135,813.191,1236.020,1311.276,71.684,173.777,222.349,15.946,1600.644,11.005


In [17]:
# Plotting functionality

n_bars = 20
period_length_sec = 3
fps = 30

def race(df, filename='EmissiesNL', pause=0, width=4, colors='dark12'):
    bcr.bar_chart_race(
        df=df,
        filename=filename+'.mp4',
        orientation='h',
        sort='desc',
        n_bars=n_bars,
        fixed_order=False,
        fixed_max=False,
        steps_per_period=fps * period_length_sec,
        interpolate_period=False,
        end_period_pause=pause,
        period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center', 'size': 8},
        # period_fmt='%B %d, %Y',
        period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                        's': f'Top {n_bars}: {v.nlargest(n_bars).sum()//1000:.0f} Mton CO2',
                                        'ha': 'right', 'size': 8},
        period_template='Jaar: {x:.0f}', 
        
        period_length=period_length_sec * 1000,
        
        perpendicular_bar_func=None, 
        # colors='dark12',
        colors=colors,
        title={
            'label': f'Per {filename} (Kton CO2 jaarlijks)',
            'size': 8,
            'color': 'black',
            'loc': 'left',
            'pad': 8
        },
        
        bar_size=.95, 
        bar_textposition='inside',
        bar_texttemplate='{x:.0f}', 
        bar_label_font=6, 
        tick_label_font=6, 
        tick_template='{x:,.0f}',
        shared_fontdict=None, 
        scale='linear', 
        fig=None, 
        writer=None, 
        bar_kwargs={'alpha': .7},
        fig_kwargs={'figsize': (width, 4), 'dpi': 300},
        filter_column_colors=False
    )

In [18]:
# rename vergunning to installatie
renamed_emissions = map_columns(filtered_emissions, 'Installatie')
grouped_emissions = renamed_emissions.groupby(renamed_emissions.columns, axis=1).sum()

# this code block is a bit convoluted, but necessary to give the bars the right color
# if it gives an error, make sure that the missing values are added to colors.csv
cols = grouped_emissions.iloc[0,:].T.sort_values(ascending=False).index
grouped_emissions = grouped_emissions[cols]
type_to_color = {'staal':'grey',
                 'chemie': 'blue',
                 'kolen':'black',
                 'raffinaderij':'yellow',
                 'gas':'white',
                 'kerncentrale':'red'}

order = set()
colors = []
for r in range(len(grouped_emissions)):
    cols = grouped_emissions.iloc[r,:].T.sort_values(ascending=False).index[:n_bars]
    for c in cols:
        order.add(c)
color_mapping = pd.read_csv('colors.csv', sep=';')
mapping = {k:v for k,v in zip(color_mapping['Naam'], color_mapping['Type'])}

for c in grouped_emissions.columns:
    if c in order:
        print(c, mapping[c], type_to_color[mapping[c]])
        colors.append(type_to_color[mapping[c]])
    else:
        colors.append('purple')

Uniper Centrale Maasvlakte kolen black
Tata Steel IJmuiden bv staal grey
Amercentrale kolen black
Chemelot chemie blue
Shell Nederland Chemie B.V. Pernis raffinaderij yellow
Vattenfall Centrale Hemweg kolen black
Dow Benelux B.V. chemie blue
Vattenfall Velsen gas white
Yara Sluiskil B.V. chemie blue
ENGIE Energie Centrale Gelderland kolen black
Shell Nederland Chemie B.V. Moerdijk chemie blue
EPZ Conventional Operations kolen black
ESSO Raffinaderij Rotterdam raffinaderij yellow
Vattenfall IJmond gas white
BP Raffinaderij Rotterdam B.V. raffinaderij yellow
ENGIE Eemscentrale gas white
Zeeland Refinery N.V. raffinaderij yellow
ENGIE Maximacentrale gas white
Pergen VOF chemie blue
Sloe Centrale B.V. gas white
Vattenfall Centrale Diemen gas white
Vattenfall Eemsmond gas white
Energie Productie Clauscentrale gas white
Enecogen gas white
Power Plant Rotterdam B.V. gas white
RWE Eemshaven Centrale kolen black


In [19]:
# make animation for installaties
race(grouped_emissions, 'installatie', pause=0, width=6, colors=colors)

In [221]:
# Now per company
renamed_emissions = map_columns(filtered_emissions, 'Bedrijf')
grouped_emissions = renamed_emissions.groupby(renamed_emissions.columns, axis=1).sum()



Sommige installaties zijn joint ventures, daar houden we rekening mee

In [222]:
grouped_emissions['Total'] += round(grouped_emissions['Zeeland Refinery N.V.'] * 0.55)
grouped_emissions['LUKoil'] = round(grouped_emissions['Zeeland Refinery N.V.'] * 0.45)
grouped_emissions.drop('Zeeland Refinery N.V.', axis=1, inplace=True)


In [223]:
grouped_emissions['PZEM'] = round(grouped_emissions['Sloe Centrale B.V.'] * 0.50)
grouped_emissions['EDF'] = round(grouped_emissions['Sloe Centrale B.V.'] * 0.50)
grouped_emissions.drop('Sloe Centrale B.V.', axis=1, inplace=True)


In [224]:
grouped_emissions['Eneco'] += round(grouped_emissions['Enecogen'] * 0.50)
grouped_emissions['Castleton'] += round(grouped_emissions['Enecogen'] * 0.50)
grouped_emissions.drop('Enecogen', axis=1, inplace=True)
 

In [225]:
# make animation for bedrijven
race(grouped_emissions, 'bedrijf', pause=0, width=4, colors=['red'])

In [21]:
widthinstallatie = 1800
widthbedrijf = 1200
overlap = 100
total = widthinstallatie + widthbedrijf - overlap
height = 1200

In [22]:
# Concat the two output files with ffmpeg in terminal
# make sure output file is not in directory otherwise doesnt work from notebook
# nullsrc is output size
# the two scales are input sizes
# the overlay is where number 2 should start

f'ffmpeg -i installatie.mp4 -i bedrijf.mp4 -filter_complex " nullsrc=size={total}x{height} [base];[0:v] setpts=PTS-STARTPTS, scale={widthinstallatie}x{height} [upperleft]; [1:v] setpts=PTS-STARTPTS, scale={widthbedrijf}x{height} [upperright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=x={widthinstallatie-overlap}" -c:v libx264 total.mp4'

'ffmpeg -i installatie.mp4 -i bedrijf.mp4 -filter_complex " nullsrc=size=2900x1200 [base];[0:v] setpts=PTS-STARTPTS, scale=1800x1200 [upperleft]; [1:v] setpts=PTS-STARTPTS, scale=1200x1200 [upperright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=x=1700" -c:v libx264 total.mp4'

For further analysis

In [226]:
total = grouped_emissions.sum(axis=0).sort_values(ascending=False)
annual = grouped_emissions.sum(axis=1)
last_year = grouped_emissions.iloc[-1,:].T.sort_values(ascending=False)